In [ ]:
import numpy as np
import librosa
import pygame
from pygame.locals import *

target_frequency = (158, 365)  # 包括中央C的頻率範圍
filename = r"C:\Users\mason\Desktop\作業\科技系\教育大數據專題製作\EDU-Data\主程式(視覺化+偏移)\一閃一閃小星星 (Twinkle Twinkle Little Star) 鋼琴教學 (MEDIUM) (mp3cut.net).wav"

# 載入音頻檔案
audio, sr = librosa.load(filename, sr=None)

# 執行短時傅立葉變換    
stft = np.abs(librosa.stft(audio, hop_length=512, n_fft=2048 * 8))  

# 初始化 pygame
pygame.init()

# 設置視窗大小
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))

# 設定震幅的最大值和最小值
max_amplitude = np.max(stft)
min_amplitude = np.min(stft)

# 定義每一幀的長度（以毫秒為單位）
frame_duration_ms = 10

# 設置時鐘對象
clock = pygame.time.Clock()

# 設置音頻
#pygame.mixer.init()
#pygame.mixer.music.load(filename)

# 開始播放音頻
#pygame.mixer.music.play()

time_data = []  # 時間數據
frequency_data = []  # 頻率數據
amplitude_data = []  # 振幅數據


# 设置刻度相关参数
scale_length = 20  # 刻度线长度
scale_spacing = 50  # 刻度线间距
scale_start = 50  # 刻度线起始位置
scale_end = screen_height - 50  # 刻度线结束位置（从下到上）
font = pygame.font.Font(None, 24)  # 字体和大小

# 遍历 stft 的每一帧，将其转换为视频中的一帧并播放
for i in range(stft.shape[1]):  # 使用 stft.shape[1] 作为时间轴的大小
    # 在这里添加你的 stft[:, i] 到视频中的处理代码
    # 例如，你可以根据 stft[:, i] 绘制一幅图像并显示在屏幕上
    # 这里简单地将 stft 的每一列作为一个折线图显示在屏幕上

    screen.fill((255, 255, 255))  # 清空屏幕，填充为白色

    # 缩放震幅到屏幕范围内
    scaled_stft = ((stft[:, i] - min_amplitude) / (max_amplitude - min_amplitude)) * screen_height-50
    
    # 将 stft 的每一列作为一个折线图显示在屏幕上
    points = [(j, screen_height - int(scaled_stft[j])) for j in range(stft.shape[0])]  # 构建折线图的点
    pygame.draw.lines(screen, (255, 0, 0), False, points, 4)  # 绘制折线图

    for y in range(scale_start, screen_height, scale_spacing):
        # 绘制刻度线
        pygame.draw.line(screen, (0, 0, 0), (scale_length, y), (0, y), 2)  # 左侧刻度线

        # 获取振幅值并将其转换为整数
        amplitude_value = int((scale_end - y) * (max_amplitude - min_amplitude) / (scale_end - scale_spacing) + min_amplitude)

        # 绘制振幅值文本
        text = font.render(str(amplitude_value), True, (0, 0, 0))  # 将文本渲染为图像
        text_rect = text.get_rect()  # 获取文本图像的矩形
        text_rect.center = (scale_length + 30, y)  # 设置文本图像的中心位置
        screen.blit(text, text_rect)  # 在屏幕上绘制文本图像

    # 更新屏幕
    pygame.display.flip()

    # 控制帧率
    clock.tick(1000 / frame_duration_ms)

    # 监听事件，例如点击关闭按钮等
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            exit()

    # 找出每个窗口的最高振幅音
    max_frequency_index = np.argmax(stft[:, i])  # 找出当前窗口最高振幅的索引
    max_frequency = librosa.fft_frequencies(sr=sr, n_fft=2048*8)[max_frequency_index]  # 将索引转换为频率
    max_amplitude = stft[max_frequency_index, i]  # 获取最高振幅
    if max_amplitude < 3:
        max_amplitude = 1
    time_data.append(i+1),frequency_data.append(max_frequency),amplitude_data.append(max_amplitude)
    print(f"窗口{i+1}：最高震幅音频率为{max_frequency} Hz，最高震幅音的振幅为{max_amplitude}")
pygame.quit()

In [2]:
import numpy as np


# 將最大振幅的頻率和振幅值組合成一個二維矩陣
max_freq_amp_matrix = np.vstack((frequency_data, amplitude_data))

# 轉置矩陣以使其與時間步長對齊
max_freq_amp_matrix = max_freq_amp_matrix.T

# 將 max_freq_amp_matrix 和 time_data 合併成一個二維矩陣
stft_data_matrix = np.concatenate((max_freq_amp_matrix, np.expand_dims(time_data, axis=1)), axis=1)

# 使用 iSTFT 將 STFT 數據轉換為時域信號
istft_signal = librosa.istft(stft_data_matrix, hop_length=512, win_length=8192,n_fft=8192)

# 打印输出的时域信号的长度

from IPython.display import Audio 
Audio(istft_signal,rate=44100)